In [21]:
import pandas as pd
import numpy as np

In [2]:
df_sam_pat = pd.read_csv('../data/samstein/data_clinical_patient.txt', error_bad_lines=False, 
                  skiprows = [0,1,2,3], low_memory=False, sep='\t')
df_sam_clin = pd.read_csv('../data/samstein/data_clinical_sample.txt', error_bad_lines=False, 
                  skiprows = [0,1,2,3], low_memory=False, sep='\t')
df_sam_mut = pd.read_csv('../data/samstein/data_mutations_extended.txt', error_bad_lines=False, 
                  low_memory=False, sep='\t')

In [3]:
df_sam_pat.head()

,PATIENT_ID,TMB_SCORE,SEX,OS_MONTHS,OS_STATUS,AGE_GROUP,DRUG_TYPE
0,P-0000057,5.580000,Female,0,DECEASED,31-50,PD-1/PDL-1
1,P-0000062,6.691462,Male,1,DECEASED,>71,PD-1/PDL-1
2,P-0000063,16.728656,Male,42,LIVING,61-70,PD-1/PDL-1
3,P-0000071,11.152437,Male,43,LIVING,61-70,PD-1/PDL-1
4,P-0000082,1.120000,Male,57,LIVING,50-60,PD-1/PDL-1


In [4]:
df_sam_mut['Hugo_Symbol'].value_counts()

TP53      834
TERT      572
CDKN2A    374
KMT2D     356
APC       260
         ... 
TCEB1       1
GTF2I       1
RRAGC       1
FIP1L1      1
RRAS        1
Name: Hugo_Symbol, Length: 469, dtype: int64

In [5]:
def get_status_label(os_months, os_status, threshold):
    if (os_months<threshold) & (os_status=='DECEASED'):
        status_label=0
    elif (os_months>=3):
        status_label=1
    else:
        status_label=np.nan
#     elif os_status=='DECEASED':
#             status_label=0
#     else:
#         status_label=np.nan
    return status_label

In [6]:
df_sam_pat['OS_MONTHS'].median()

11.0

In [7]:
df_sam_pat['OS_MONTHS'].quantile(0.1)

2.0

In [8]:
df_sam_pat.DRUG_TYPE.value_counts()

PD-1/PDL-1    1307
Combo          255
CTLA4           99
Name: DRUG_TYPE, dtype: int64

In [9]:
df_sam_pat['label'] = df_sam_pat.apply(lambda row: get_status_label(row.OS_MONTHS, row.OS_STATUS, 11), axis=1)

In [10]:
df_sam_pat1 = df_sam_pat[df_sam_pat['DRUG_TYPE']=='PD-1/PDL-1']

In [11]:
len(df_sam_pat1)

1307

In [12]:
df_sam_pat1.head(10)

,PATIENT_ID,TMB_SCORE,SEX,OS_MONTHS,OS_STATUS,AGE_GROUP,DRUG_TYPE,label
0,P-0000057,5.580000,Female,0,DECEASED,31-50,PD-1/PDL-1,0.0
1,P-0000062,6.691462,Male,1,DECEASED,>71,PD-1/PDL-1,0.0
2,P-0000063,16.728656,Male,42,LIVING,61-70,PD-1/PDL-1,1.0
3,P-0000071,11.152437,Male,43,LIVING,61-70,PD-1/PDL-1,1.0
4,P-0000082,1.120000,Male,57,LIVING,50-60,PD-1/PDL-1,1.0
5,P-0000088,13.382924,Male,12,DECEASED,61-70,PD-1/PDL-1,1.0
7,P-0000121,3.345731,Male,4,LIVING,31-50,PD-1/PDL-1,1.0
8,P-0000165,5.580000,Female,1,DECEASED,61-70,PD-1/PDL-1,0.0
9,P-0000184,8.921950,Male,8,LIVING,61-70,PD-1/PDL-1,1.0
11,P-0000208,2.230000,Female,13,DECEASED,61-70,PD-1/PDL-1,1.0


In [13]:
len(df_sam_pat1.dropna(subset=['label']))

1254

In [14]:
df_sam_pat1['label'].value_counts()

1.0    773
0.0    481
Name: label, dtype: int64

In [15]:
ptid_label_dict = df_sam_pat[['PATIENT_ID', 'label']].set_index('PATIENT_ID').to_dict()['label']

In [16]:
df_sam_clin.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,SAMPLE_TYPE,SAMPLE_CLASS,METASTATIC_SITE,PRIMARY_SITE,CANCER_TYPE_DETAILED,GENE_PANEL,SAMPLE_COVERAGE,TUMOR_PURITY,ONCOTREE_CODE,INSTITUTE,SOMATIC_STATUS,AGE_AT_SEQ_REPORT
0,P-0000057,P-0000057-T01-IM3,Breast Cancer,Primary,Tumor,Not Applicable,Breast,Breast Mixed Ductal and Lobular Carcinoma,IMPACT341,835,25,MDLC,MSKCC,Matched,41.0
1,P-0000062,P-0000062-T01-IM3,Esophagogastric Cancer,Primary,Tumor,Not Applicable,Esophagus,Adenocarcinoma of the Gastroesophageal Junction,IMPACT341,1176,30,GEJ,MSKCC,Matched,80.0
2,P-0000063,P-0000063-T01-IM3,Bladder Cancer,Primary,Tumor,Not Applicable,Bladder,Bladder Urothelial Carcinoma,IMPACT341,900,70,BLCA,MSKCC,Matched,62.0
3,P-0000071,P-0000071-T01-IM3,Bladder Cancer,Primary,Tumor,Not Applicable,Bladder,Bladder Urothelial Carcinoma,IMPACT341,795,30,BLCA,MSKCC,Matched,66.0
4,P-0000082,P-0000082-T01-IM3,Non-Small Cell Lung Cancer,Primary,Tumor,Not Applicable,Lung,Lung Adenocarcinoma,IMPACT341,905,NaN,LUAD,MSKCC,Matched,61.0


In [17]:
df_sam_clin.columns

Index(['PATIENT_ID', 'SAMPLE_ID', 'CANCER_TYPE', 'SAMPLE_TYPE', 'SAMPLE_CLASS',
       'METASTATIC_SITE', 'PRIMARY_SITE', 'CANCER_TYPE_DETAILED', 'GENE_PANEL',
       'SAMPLE_COVERAGE', 'TUMOR_PURITY', 'ONCOTREE_CODE', 'INSTITUTE',
       'SOMATIC_STATUS', 'AGE_AT_SEQ_REPORT'],
      dtype='object')

In [18]:
df_sam_clin['CANCER_TYPE'].value_counts()

Non-Small Cell Lung Cancer    350
Melanoma                      320
Bladder Cancer                215
Renal Cell Carcinoma          151
Head and Neck Cancer          139
Esophagogastric Cancer        126
Glioma                        117
Colorectal Cancer             110
Cancer of Unknown Primary      88
Breast Cancer                  44
Skin Cancer, Non-Melanoma       1
Name: CANCER_TYPE, dtype: int64

In [20]:
df_sam_clin[df_sam_clin['CANCER_TYPE']=='Non-Small Cell Lung Cancer']['CANCER_TYPE_DETAILED'].value_counts()

Lung Adenocarcinoma                                 271
Lung Squamous Cell Carcinoma                         45
Poorly Differentiated Non-Small Cell Lung Cancer     13
Large Cell Neuroendocrine Carcinoma                   8
Non-Small Cell Lung Cancer                            8
Lung Adenosquamous Carcinoma                          2
Sarcomatoid Carcinoma of the Lung                     2
Pleomorphic Carcinoma of the Lung                     1
Name: CANCER_TYPE_DETAILED, dtype: int64

In [19]:
cancer_type_dict = df_sam_clin[['PATIENT_ID', 'CANCER_TYPE']].set_index('PATIENT_ID').to_dict()['CANCER_TYPE']

In [22]:
df_sam_clin1 = df_sam_clin[df_sam_clin['CANCER_TYPE_DETAILED']=='Lung Adenocarcinoma']

In [23]:
df_sam_clin1.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,SAMPLE_TYPE,SAMPLE_CLASS,METASTATIC_SITE,PRIMARY_SITE,CANCER_TYPE_DETAILED,GENE_PANEL,SAMPLE_COVERAGE,TUMOR_PURITY,ONCOTREE_CODE,INSTITUTE,SOMATIC_STATUS,AGE_AT_SEQ_REPORT
4,P-0000082,P-0000082-T01-IM3,Non-Small Cell Lung Cancer,Primary,Tumor,Not Applicable,Lung,Lung Adenocarcinoma,IMPACT341,905,NaN,LUAD,MSKCC,Matched,61.0
8,P-0000165,P-0000165-T01-IM3,Non-Small Cell Lung Cancer,Primary,Tumor,Not Applicable,Lung,Lung Adenocarcinoma,IMPACT341,795,40,LUAD,MSKCC,Unmatched,67.0
10,P-0000205,P-0000205-T01-IM3,Non-Small Cell Lung Cancer,Metastasis,Tumor,Liver,Lung,Lung Adenocarcinoma,IMPACT341,1214,90,LUAD,MSKCC,Unmatched,56.0
11,P-0000208,P-0000208-T01-IM3,Non-Small Cell Lung Cancer,Primary,Tumor,Not Applicable,Lung,Lung Adenocarcinoma,IMPACT410,1346,70,LUAD,MSKCC,Matched,69.0
30,P-0000458,P-0000458-T01-IM3,Non-Small Cell Lung Cancer,Primary,Tumor,Not Applicable,Lung,Lung Adenocarcinoma,IMPACT341,1201,20,LUAD,MSKCC,Matched,66.0


In [24]:
df_sam_clin1['CANCER_TYPE_DETAILED'].value_counts()

Lung Adenocarcinoma    271
Name: CANCER_TYPE_DETAILED, dtype: int64

In [25]:
df_sam_clin1['SAMPLE_TYPE'].value_counts()

Metastasis    142
Primary       129
Name: SAMPLE_TYPE, dtype: int64

In [26]:
patient_id_dict = df_sam_clin[['SAMPLE_ID', 'PATIENT_ID']].set_index('SAMPLE_ID').to_dict()['PATIENT_ID']

In [27]:
ptid_lung_cancer = df_sam_clin1['PATIENT_ID'].unique()

In [28]:
df_sam_mut.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
0,PIK3CB,5291.0,MSKCC,GRCh37,3,138374293,138374293,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TP53,7157.0,MSKCC,GRCh37,17,7577539,7577539,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TP53,7157.0,MSKCC,GRCh37,17,7577099,7577099,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PIK3C2G,5288.0,MSKCC,GRCh37,12,18435667,18435667,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AXIN1,8312.0,MSKCC,GRCh37,16,347756,347756,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_sam_mut1 = df_sam_mut[['Hugo_Symbol',  'NCBI_Build', 'Chromosome',
       'Start_Position', 'End_Position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Tumor_Sample_Barcode', 'HGVSp_Short',
        'Protein_position', 'Codons', 't_ref_count', 't_alt_count']]

In [30]:
df_sam_mut2 = df_sam_mut1.dropna(subset=['Codons'])

In [31]:
df_sam_mut3 = df_sam_mut1[~(df_sam_mut1['Variant_Classification']=='Silent')]

In [32]:
df_sam_mut3.head()

,Hugo_Symbol,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,HGVSp_Short,Protein_position,Codons,t_ref_count,t_alt_count
0,PIK3CB,GRCh37,3,138374293,138374293,+,Missense_Mutation,SNP,C,C,T,P-0024731-T01-IM6,p.E1051K,1051.0,Gaa/Aaa,158,181
1,TP53,GRCh37,17,7577539,7577539,+,Missense_Mutation,SNP,G,G,A,P-0024731-T01-IM6,p.R248W,248.0,Cgg/Tgg,126,61
2,TP53,GRCh37,17,7577099,7577099,+,Missense_Mutation,SNP,C,C,T,P-0024731-T01-IM6,p.R280K,280.0,aGa/aAa,170,118
3,PIK3C2G,GRCh37,12,18435667,18435667,+,Missense_Mutation,SNP,G,G,A,P-0024731-T01-IM6,p.E218K,218.0,Gaa/Aaa,91,54
4,AXIN1,GRCh37,16,347756,347756,+,Missense_Mutation,SNP,C,C,T,P-0024731-T01-IM6,p.A584T,584.0,Gct/Act,124,26


In [33]:
def get_patient_ids(tumor_sample_barcode):
    try:
        ptid=patient_id_dict[tumor_sample_barcode]
    except:
        ptid=np.nan
    return ptid

In [34]:
df_sam_mut3['PATIENT_ID'] = df_sam_mut3['Tumor_Sample_Barcode'].map(lambda x: get_patient_ids(x))

/home/adnan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
df_sam_mut4 = df_sam_mut3[df_sam_mut3['PATIENT_ID'].isin(ptid_lung_cancer)]
#df_sam_mut4 = df_sam_mut3

In [36]:
df_sam_mut4.head()

,Hugo_Symbol,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,HGVSp_Short,Protein_position,Codons,t_ref_count,t_alt_count,PATIENT_ID
272,RFWD2,GRCh37,1,176132107,176132107,+,Missense_Mutation,SNP,C,C,A,P-0000082-T01-IM3,p.Q220H,220.0,caG/caT,872,248,P-0000082
273,REL,GRCh37,2,61121637,61121637,+,Missense_Mutation,SNP,G,G,T,P-0000082-T01-IM3,p.G87C,87.0,Ggc/Tgc,530,157,P-0000082
274,IRS1,GRCh37,2,227660531,227660531,+,Missense_Mutation,SNP,A,A,G,P-0000082-T01-IM3,p.I975T,975.0,aTt/aCt,547,171,P-0000082
275,NOTCH1,GRCh37,9,139405633,139405635,+,Missense_Mutation,ONP,AAG,AAG,CTC,P-0000082-T01-IM3,p.S852_F853delinsRS,852.0,agCTTc/agGAGc,629,144,P-0000082
276,NOTCH1,GRCh37,9,139413143,139413143,+,Missense_Mutation,SNP,G,G,C,P-0000082-T01-IM3,p.S333R,333.0,agC/agG,546,213,P-0000082


In [37]:
def get_af(row):
    alt_count = row.t_alt_count
    ref_count = row.t_ref_count
    try:
        af = alt_count/(alt_count+ref_count)
    except:
        af = 0
    return af

In [38]:
df_sam_mut4['af'] = df_sam_mut4.apply(lambda row: get_af(row), axis=1)

/home/adnan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Creating data for ML

In [ ]:
#df_allen_mut5 = df_allen_mut5[df_allen_mut5['Variant_Classification']=='Missense_Mutation'] 

In [ ]:
10	KEAP1	1.524761	0.014914	2.434457	62
13	STK11	1.477622	0.023934	2.258190	66
17	PBRM1	2.311227	0.000464	3.500881	17
43	ZFHX3	0.344114	0.021066	-2.306792	21
72	MET	0.237527	0.035938	-2.097630	11
116	NTRK3	0.373096	0.044661	-2.007835	18

In [39]:
len(df_sam_mut4['PATIENT_ID'].unique())

266

<font color=red> Creating data according to New genes </font>

In [40]:
data_dict = {}
for ptid in df_sam_mut4['PATIENT_ID'].unique():
    df_tmp1 = df_sam_mut4[df_sam_mut4['PATIENT_ID']==ptid]
    
    KEAP1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1'])
    STK11_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='STK11'])
    PBRM1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='PBRM1'])
    ZFHX3_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='ZFHX3'])
    MET_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='MET'])
    TP53_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='TP53'])
    
    tmb = len(df_tmp1)
    KEAP1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1']['af'].max()
    STK11_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='STK11']['af'].max()
    PBRM1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='PBRM1']['af'].max()
    ZFHX3_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='ZFHX3']['af'].max()
    MET_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='MET']['af'].max()
    TP53_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='TP53']['af'].max()
    
    data_dict_tmp ={}
    data_dict_tmp['KEAP1_count'] = KEAP1_count
    data_dict_tmp['STK11_count'] = STK11_count
    data_dict_tmp['PBRM1_count'] = PBRM1_count
    data_dict_tmp['ZFHX3_count'] = ZFHX3_count
    data_dict_tmp['MET_count'] = MET_count
    data_dict_tmp['TP53_count'] = TP53_count
    
    data_dict_tmp['tmb'] = tmb
    data_dict_tmp['KEAP1_af'] = KEAP1_af
    data_dict_tmp['STK11_af'] = STK11_af
    data_dict_tmp['PBRM1_af'] = PBRM1_af
    data_dict_tmp['ZFHX3_af'] = ZFHX3_af
    data_dict_tmp['MET_af'] = MET_af
    data_dict_tmp['TP53_af'] = TP53_af
    
    data_dict[ptid] = data_dict_tmp

<font color=red> uncomment and run the following code to create data according to Bradley 4 genes </font>

In [ ]:
# data_dict = {}
# for ptid in df_sam_mut4['PATIENT_ID'].unique():
#     df_tmp1 = df_sam_mut4[df_sam_mut4['PATIENT_ID']==ptid]
#     TP53_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='TP53'])
#     KEAP1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1'])
#     SMARCA4_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='SMARCA4'])
#     STK11_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='STK11'])
#     tmb = len(df_tmp1)
#     TP53_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='TP53']['af'].max()
#     KEAP1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1']['af'].max()
#     SMARCA4_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='SMARCA4']['af'].max()
#     STK11_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='STK11']['af'].max()
    
#     data_dict_tmp ={}
#     data_dict_tmp['TP53_count'] = TP53_count
#     data_dict_tmp['KEAP1_count'] = KEAP1_count
#     data_dict_tmp['SMARCA4_count'] = SMARCA4_count
#     data_dict_tmp['STK11_count'] = STK11_count
#     data_dict_tmp['tmb'] = tmb
#     data_dict_tmp['TP53_af'] = TP53_af
#     data_dict_tmp['KEAP1_af'] = KEAP1_af
#     data_dict_tmp['SMARCA4_af'] = SMARCA4_af
#     data_dict_tmp['STK11_af'] = STK11_af
    
#     data_dict[ptid] = data_dict_tmp

In [41]:
df_test_data = pd.DataFrame.from_dict(data_dict, orient='index').reset_index()

In [42]:
df_test_data = df_test_data.fillna(0)

In [43]:
df_test_data = df_test_data.rename(columns={'index':'PtID'})

In [44]:
df_test_data.head(10)

,PtID,KEAP1_count,STK11_count,PBRM1_count,ZFHX3_count,MET_count,TP53_count,tmb,KEAP1_af,STK11_af,PBRM1_af,ZFHX3_af,MET_af,TP53_af
0,P-0000082,0,0,0,0,0,1,12,0.000000,0.000000,0.000000,0.000000,0.000000,0.372549
1,P-0002766,1,1,0,0,0,0,6,0.134921,0.176849,0.000000,0.000000,0.000000,0.000000
2,P-0007083,0,0,1,0,0,1,16,0.000000,0.000000,0.346667,0.000000,0.000000,0.375536
3,P-0015078,0,0,0,0,0,0,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,P-0015214,0,0,0,1,0,1,28,0.000000,0.000000,0.000000,0.427619,0.000000,0.440594
5,P-0015222,1,1,0,0,0,1,9,0.263425,0.207831,0.000000,0.000000,0.000000,0.285270
6,P-0015259,0,0,0,0,0,0,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,P-0015348,0,0,0,0,0,1,12,0.000000,0.000000,0.000000,0.000000,0.000000,0.352507
8,P-0015556,1,0,0,0,0,0,5,0.151923,0.000000,0.000000,0.000000,0.000000,0.000000
9,P-0015593,0,0,0,0,1,1,5,0.000000,0.000000,0.000000,0.000000,0.400309,0.143135


In [45]:
df_test_data['label'] = df_test_data['PtID'].map(lambda x: ptid_label_dict[x])

In [46]:
df_test_data1 = df_test_data.dropna(subset=['label'])

In [47]:
df_test_data1.label.value_counts()

1.0    141
0.0    118
Name: label, dtype: int64

In [ ]:
df_test_data['cancer_type'] = df_test_data['PtID'].map(lambda x: cancer_type_dict[x])

#### one hot encoding

In [ ]:
df_test_data1 = pd.concat([df_test_data,pd.get_dummies(df_test_data['cancer_type'], prefix='cancer_type')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_test_data1.drop(['cancer_type'],axis=1, inplace=True)

In [251]:
df_test_data1.columns

Index(['PtID', 'KEAP1_count', 'STK11_count', 'PBRM1_count', 'ZFHX3_count',
       'MET_count', 'TP53_count', 'tmb', 'KEAP1_af', 'STK11_af', 'PBRM1_af',
       'ZFHX3_af', 'MET_af', 'TP53_af', 'label'],
      dtype='object')

In [252]:
df_test_data1.label.value_counts()

1.0    141
0.0    118
Name: label, dtype: int64

In [253]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import PCA as sklearnPCA
from sklearn import preprocessing
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from statistics import mean
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")
import random 
random.seed(1234)

In [254]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import neighbors
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [255]:
df_test_data2 = df_test_data1.sample(frac=1).reset_index(drop=True)

In [256]:
df_test_data2 = (df_test_data1.dropna())

In [257]:
df_test_data2.label.value_counts()

1.0    141
0.0    118
Name: label, dtype: int64

#### Oversampling using smote

In [258]:
X = df_test_data1[['tmb', 'KEAP1_count', 'STK11_count', 'PBRM1_count', 'ZFHX3_count',
                   'MET_count', 'TP53_count', 'KEAP1_af', 'STK11_af', 'PBRM1_af',
                   'ZFHX3_af', 'MET_af', 'TP53_af']]

In [259]:
scaler = preprocessing.MinMaxScaler()
scaler_mod = scaler.fit(X)
scaled_X = scaler_mod.transform(X)
df_scaled_X = pd.DataFrame(scaled_X, columns=X.columns)

In [260]:
X = df_scaled_X.values
Y = df_test_data2['label'].values

In [261]:
def balance_smote(X, Y):
    from imblearn.over_sampling import SMOTE
    from collections import Counter
    #X = df.drop('label', axis=1)
    #Y = df.label
    print('Original data shapes:', X.shape, Y.shape)
    
    smoX, smoY = X, Y
    c = Counter(smoY)
    while (min(c.values()) < max(c.values())):  # check if all classes are balanced, if not balance the first minority class
        smote = SMOTE(ratio="auto", kind='regular')
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            smoX, smoY = smote.fit_sample(smoX, smoY)
        c = Counter(smoY)
    
    print('Balanced data shapes:', smoX.shape, smoY.shape)
    return smoX, smoY

In [262]:
X, Y = balance_smote(X, Y)

Original data shapes: (259, 13) (259,)
Balanced data shapes: (282, 13) (282,)


In [263]:
model1 = GradientBoostingClassifier()

model2 = RandomForestClassifier(max_depth=7, n_estimators=10, max_features=3)


#bagging classifier
model3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, max_features=4)

#multi layer perceptron
#model8 = MLPClassifier(alpha=1, max_iter=1000)

model4 = AdaBoostClassifier()



model5 = RandomForestClassifier(max_depth=7, n_estimators=150, max_features=3)

models =[model1, model2,model3, model4, model5]

In [264]:
from sklearn.model_selection import cross_val_score
clf = GradientBoostingClassifier()
scores = cross_val_score(clf, X, Y, cv=5)

In [265]:
scores.mean()

0.6379310344827587

In [266]:
#evaluating all models using cross validation
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import balanced_accuracy_score
random.seed(1234)

skf = StratifiedKFold(n_splits=8, shuffle=True)
#skf = KFold(n_splits = 7, shuffle = True, random_state= 0)
models_accuracy=[]
models_roc_auc=[]

output = {}
i=1
for model in models:
    accuracy_list=[]
    roc_auc_list=[]
    
    for train_index, test_index in skf.split(X, Y):
        model.fit(X[train_index], Y[train_index])
        y_pred = model.predict(X[test_index])
        y_prob = model.predict_proba(X[test_index])[:,1]
        
        accuracy = balanced_accuracy_score(Y[test_index], y_pred)
        accuracy_list.append(accuracy)
        
        roc_auc = roc_auc_score(Y[test_index], y_prob)
        roc_auc_list.append(roc_auc)
        
    mean_accuracy=np.mean(accuracy_list)
    mean_roc_auc=np.mean(roc_auc_list)

    
    models_accuracy.append(mean_accuracy)
    models_roc_auc.append(mean_roc_auc)
    model_perf={}
    model_perf['accuracy'] = mean_accuracy
    model_perf['roc_auc'] = mean_roc_auc
    
    model_num = 'model'+str(i)
    output[model_num]= model_perf
    i=i+1

In [267]:
output

{'model1': {'accuracy': 0.6875, 'roc_auc': 0.7437964564910932},
 'model2': {'accuracy': 0.6560457516339869, 'roc_auc': 0.7068355119825708},
 'model3': {'accuracy': 0.6233660130718954, 'roc_auc': 0.6853907685078389},
 'model4': {'accuracy': 0.6158088235294117, 'roc_auc': 0.6733968505702934},
 'model5': {'accuracy': 0.673406862745098, 'roc_auc': 0.7134295303088556}}

In [268]:
model_gbc = GradientBoostingClassifier()
model_gbc.fit(X,Y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

## 2. Allens evaluation

In [22]:
df_allen_pat = pd.read_csv('../data/allen/data_clinical_patient.txt', error_bad_lines=False, 
                  skiprows = [0,1,2,3], low_memory=False, sep='\t')
df_allen_sam = pd.read_csv('../data/allen/data_clinical_sample.txt', error_bad_lines=False, 
                  skiprows = [0,1,2,3], low_memory=False, sep='\t')
df_allen_mut = pd.read_csv('../data/allen/data_mutations_extended.txt', error_bad_lines=False, 
                  low_memory=False, sep='\t')

In [23]:
df_allen_pat.head()

,PATIENT_ID,SEX,P16_ISH,OS_MONTHS,PFS_MONTHS,PFS_STATUS,HISTOLOGY,SUBTYPE,SMOKER,SMOKING_PACK_YEARS,RECIST_RESPONSE,AGE_START_IO,RECIST,VA_RESPONSE,ROH_RESPONSE,DRUG_TYPE,OTHER_CONCURRENT_THERAPY,OS_STATUS
0,AC_PD1-1,Male,NaN,27.01,27.01,1.0,NaN,NaN,NaN,NaN,clinical benefit,NaN,CR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING
1,luad_mskcc_2015_15,Male,NaN,8.39,8.39,1.0,adenocarcinoma,NaN,Former,34.00,stable disease,59.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING
2,luad_mskcc_2015_33,Male,NaN,1.78,1.78,0.0,adenocarcinoma,NaN,Former,10.00,no clinical benefit,64.0,PD,no clinical benefit,no clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING
3,luad_mskcc_2015_22,Female,NaN,6.48,6.48,0.0,adenocarcinoma,NaN,Former,43.75,stable disease,73.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING
4,BLADDER-15330_CCPM_0700629,Male,NaN,3.49,1.78,1.0,NaN,NaN,NaN,NaN,clinical benefit,69.0,PR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING


In [28]:
df_allen_sam[df_allen_sam['TUMOR_TYPE']=='Lung']['CANCER_TYPE_DETAILED'].value_counts()

Lung Adenocarcinoma             47
Lung Squamous Cell Carcinoma     7
Non-Small Cell Lung Cancer       2
Small Cell Lung Cancer           1
Name: CANCER_TYPE_DETAILED, dtype: int64

In [24]:
df_allen_pat['OS_STATUS'].value_counts()

DECEASED    125
LIVING      124
Name: OS_STATUS, dtype: int64

In [25]:
def get_status_label(os_months, os_status, threshold):
    if (os_months<threshold) & (os_status=='DECEASED'):
        status_label=0
    elif (os_months>=3):
        status_label=1
    else:
        status_label=np.nan
#     elif os_status=='DECEASED':
#             status_label=0
#     else:
#         status_label=np.nan
    return status_label

In [26]:
df_allen_pat['label'] = df_allen_pat.apply(lambda row: get_status_label(row.OS_MONTHS, row.OS_STATUS, 11), axis=1)

In [ ]:
df_allen_pat[df_allen_pat['PATIENT_ID']=='luad_mskcc_2015_24']

In [ ]:
df_allen_sam[df_allen_sam['PATIENT_ID']=='luad_mskcc_2015_24']

In [ ]:
df_allen_sam.head()

In [273]:
patient_id_dict = df_allen_sam[['SAMPLE_ID', 'PATIENT_ID']].set_index('SAMPLE_ID').to_dict()['PATIENT_ID']

### Clinical data

In [29]:
df_allen_pat.head()

,PATIENT_ID,SEX,P16_ISH,OS_MONTHS,PFS_MONTHS,PFS_STATUS,HISTOLOGY,SUBTYPE,SMOKER,SMOKING_PACK_YEARS,RECIST_RESPONSE,AGE_START_IO,RECIST,VA_RESPONSE,ROH_RESPONSE,DRUG_TYPE,OTHER_CONCURRENT_THERAPY,OS_STATUS,label
0,AC_PD1-1,Male,NaN,27.01,27.01,1.0,NaN,NaN,NaN,NaN,clinical benefit,NaN,CR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
1,luad_mskcc_2015_15,Male,NaN,8.39,8.39,1.0,adenocarcinoma,NaN,Former,34.00,stable disease,59.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
2,luad_mskcc_2015_33,Male,NaN,1.78,1.78,0.0,adenocarcinoma,NaN,Former,10.00,no clinical benefit,64.0,PD,no clinical benefit,no clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,NaN
3,luad_mskcc_2015_22,Female,NaN,6.48,6.48,0.0,adenocarcinoma,NaN,Former,43.75,stable disease,73.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
4,BLADDER-15330_CCPM_0700629,Male,NaN,3.49,1.78,1.0,NaN,NaN,NaN,NaN,clinical benefit,69.0,PR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0


In [275]:
df_allen_pat['HISTOLOGY'].value_counts()

cutaneous                  120
adenocarcinoma              36
occult                      14
Adenocarcinoma              11
tonsil                       6
squamous cell carcinoma      5
mucosal                      5
NSCLC NOS                    2
oropharynx                   2
Squamous Cell Carcinoma      2
Small Cell Lung Cancer       1
larynx                       1
cutaneous and eye            1
oral tongue                  1
uveal                        1
nasopharynx                  1
base of tongue               1
leiomyosarcoma               1
Name: HISTOLOGY, dtype: int64

#### filtering only lung cancer data

In [276]:
df_allen_pat1 = df_allen_pat[(df_allen_pat['HISTOLOGY']=='adenocarcinoma')|(df_allen_pat['HISTOLOGY']=='Adenocarcinoma')]

In [277]:
len(df_allen_pat1)

47

In [314]:
df_allen_pat1.head()

,PATIENT_ID,SEX,P16_ISH,OS_MONTHS,PFS_MONTHS,PFS_STATUS,HISTOLOGY,SUBTYPE,SMOKER,SMOKING_PACK_YEARS,RECIST_RESPONSE,AGE_START_IO,RECIST,VA_RESPONSE,ROH_RESPONSE,DRUG_TYPE,OTHER_CONCURRENT_THERAPY,OS_STATUS,label
1,luad_mskcc_2015_15,Male,NaN,8.39,8.39,1.0,adenocarcinoma,NaN,Former,34.00,stable disease,59.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
2,luad_mskcc_2015_33,Male,NaN,1.78,1.78,0.0,adenocarcinoma,NaN,Former,10.00,no clinical benefit,64.0,PD,no clinical benefit,no clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,NaN
3,luad_mskcc_2015_22,Female,NaN,6.48,6.48,0.0,adenocarcinoma,NaN,Former,43.75,stable disease,73.0,SD,no clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
31,luad_mskcc_2015_9,Male,NaN,14.51,14.51,0.0,adenocarcinoma,NaN,Former,80.00,clinical benefit,57.0,PR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0
32,CANSEQU01-0100425,Female,NaN,24.18,6.81,0.0,Adenocarcinoma,NaN,former,45.00,clinical benefit,NaN,PR,clinical benefit,clinical benefit,anti-PD-1/anti-PD-L1,no,LIVING,1.0


In [315]:
df_allen_pat1['CNSR'] = df_allen_pat1['OS_STATUS'].map(lambda x: 1 if x=='DECEASED' else 0)

In [316]:
os_months_dict = df_allen_pat1[['PATIENT_ID', 'OS_MONTHS']].set_index('PATIENT_ID').to_dict()['OS_MONTHS']

In [317]:
cnsr_dict = df_allen_pat1[['PATIENT_ID', 'CNSR']].set_index('PATIENT_ID').to_dict()['CNSR']

In [279]:
df_allen_pat1.label.value_counts()

1.0    30
0.0    10
Name: label, dtype: int64

In [ ]:
df_allen_pat1['RECIST_RESPONSE'].value_counts()

In [ ]:
df_allen_pat1['RECIST'].value_counts()

In [ ]:
df_allen_pat1['DRUG_TYPE'].value_counts()

In [280]:
df_allen_pat2 = df_allen_pat1[(df_allen_pat1['DRUG_TYPE']=='anti-PD-1/anti-PD-L1')]

In [281]:
#df_allen_pat3 = df_allen_pat2[~(df_allen_pat2['RECIST']=='SD')]
df_allen_pat3 = df_allen_pat2.dropna(subset=['label'])

There are total 57 patients. We are not interested in stable disease so after removing it we are left with 36 patients. One patient is one ant CTLA4 drug as well. We have removed him as well.

In [282]:
len(df_allen_pat3)

40

In [283]:
df_allen_pat3.HISTOLOGY.value_counts()

adenocarcinoma    29
Adenocarcinoma    11
Name: HISTOLOGY, dtype: int64

In [284]:
patient_ids_allen = df_allen_pat3['PATIENT_ID'].values

In [ ]:
df_allen_pat3.head()

In [ ]:
def get_label(disease_status):
    if disease_status == 'PD':
        label=0
    elif disease_status =='PR':
        label=1
    elif disease_status=='CR':
        label=1
    else:
        label = np.nan
    return label

In [ ]:
df_allen_pat3['recist_label'] = df_allen_pat3['RECIST'].map(lambda x: get_label(x))

In [ ]:
df_allen_pat3.head(10)

In [285]:
labels_dict = df_allen_pat3[['PATIENT_ID', 'label']].set_index('PATIENT_ID').to_dict()['label']

In [ ]:
labels_recist_dict = df_allen_pat3[['PATIENT_ID', 'recist_label']].set_index('PATIENT_ID').to_dict()['recist_label']

### Mutations data

In [286]:
df_allen_mut = pd.read_csv('../data/allen/data_mutations_extended.txt', error_bad_lines=False, 
                  low_memory=False, sep='\t')

In [ ]:
df_allen_mut.columns

In [287]:
df_allen_mut1 = df_allen_mut[['Hugo_Symbol',  'NCBI_Build', 'Chromosome',
       'Start_Position', 'End_Position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Tumor_Sample_Barcode', 'HGVSp_Short',
        'Protein_position', 'Codons', 't_ref_count', 't_alt_count']]

In [288]:
df_allen_mut1.Variant_Type.value_counts()

SNP    163970
DEL      2263
INS      1084
Name: Variant_Type, dtype: int64

In [ ]:
df_allen_mut2 = df_allen_mut1[df_allen_mut1['Variant_Type']=='SNP']

In [ ]:
df_allen_mut3 = df_allen_mut1.dropna(subset=['Codons'])

In [289]:
df_allen_mut4 = df_allen_mut1[~(df_allen_mut1['Variant_Classification']=='Silent')]

In [290]:
df_allen_mut4['PATIENT_ID'] = df_allen_mut4['Tumor_Sample_Barcode'].map(lambda x: patient_id_dict[x])

In [291]:
df_allen_mut5 = df_allen_mut4[df_allen_mut4['PATIENT_ID'].isin(patient_ids_allen)]

In [292]:
df_allen_mut5.head()

,Hugo_Symbol,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,HGVSp_Short,Protein_position,Codons,t_ref_count,t_alt_count,PATIENT_ID
359,PTK7,GRCh37,6,43114362,43114362,+,Missense_Mutation,SNP,C,C,G,AL4602,p.L883V,883.0,Ctc/Gtc,108,14,luad_mskcc_2015_15
360,DZANK1,GRCh37,20,18393435,18393435,+,Missense_Mutation,SNP,C,C,A,AL4602,p.R429S,429.0,agG/agT,103,11,luad_mskcc_2015_15
361,KCNH7,GRCh37,2,163361083,163361083,+,Missense_Mutation,SNP,G,G,A,AL4602,p.P333L,333.0,cCa/cTa,206,14,luad_mskcc_2015_15
362,UGT1A3,GRCh37,2,234638106,234638106,+,Nonsense_Mutation,SNP,A,A,T,AL4602,p.R112*,112.0,Aga/Tga,278,13,luad_mskcc_2015_15
363,FAM110B,GRCh37,8,59059129,59059129,+,Missense_Mutation,SNP,G,G,C,AL4602,p.E114Q,114.0,Gag/Cag,37,4,luad_mskcc_2015_15


In [293]:
def get_af(row):
    alt_count = row.t_alt_count
    ref_count = row.t_ref_count
    try:
        af = alt_count/(alt_count+ref_count)
    except:
        af = 0
    return af

In [294]:
df_allen_mut5['af'] = df_allen_mut5.apply(lambda row: get_af(row), axis=1)

#### Creating data for ML

In [295]:
df_allen_mut5.head()

,Hugo_Symbol,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,HGVSp_Short,Protein_position,Codons,t_ref_count,t_alt_count,PATIENT_ID,af
359,PTK7,GRCh37,6,43114362,43114362,+,Missense_Mutation,SNP,C,C,G,AL4602,p.L883V,883.0,Ctc/Gtc,108,14,luad_mskcc_2015_15,0.114754
360,DZANK1,GRCh37,20,18393435,18393435,+,Missense_Mutation,SNP,C,C,A,AL4602,p.R429S,429.0,agG/agT,103,11,luad_mskcc_2015_15,0.096491
361,KCNH7,GRCh37,2,163361083,163361083,+,Missense_Mutation,SNP,G,G,A,AL4602,p.P333L,333.0,cCa/cTa,206,14,luad_mskcc_2015_15,0.063636
362,UGT1A3,GRCh37,2,234638106,234638106,+,Nonsense_Mutation,SNP,A,A,T,AL4602,p.R112*,112.0,Aga/Tga,278,13,luad_mskcc_2015_15,0.044674
363,FAM110B,GRCh37,8,59059129,59059129,+,Missense_Mutation,SNP,G,G,C,AL4602,p.E114Q,114.0,Gag/Cag,37,4,luad_mskcc_2015_15,0.097561


In [296]:
data_dict = {}
for ptid in df_allen_mut5['PATIENT_ID'].unique():
    df_tmp1 = df_allen_mut5[df_allen_mut5['PATIENT_ID']==ptid]
    
    KEAP1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1'])
    STK11_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='STK11'])
    PBRM1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='PBRM1'])
    ZFHX3_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='ZFHX3'])
    MET_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='MET'])
    TP53_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='TP53'])
    
    tmb = len(df_tmp1)
    KEAP1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1']['af'].max()
    STK11_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='STK11']['af'].max()
    PBRM1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='PBRM1']['af'].max()
    ZFHX3_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='ZFHX3']['af'].max()
    MET_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='MET']['af'].max()
    TP53_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='TP53']['af'].max()
    
    data_dict_tmp ={}
    data_dict_tmp['KEAP1_count'] = KEAP1_count
    data_dict_tmp['STK11_count'] = STK11_count
    data_dict_tmp['PBRM1_count'] = PBRM1_count
    data_dict_tmp['ZFHX3_count'] = ZFHX3_count
    data_dict_tmp['MET_count'] = MET_count
    data_dict_tmp['TP53_count'] = TP53_count
    
    data_dict_tmp['tmb'] = tmb
    data_dict_tmp['KEAP1_af'] = KEAP1_af
    data_dict_tmp['STK11_af'] = STK11_af
    data_dict_tmp['PBRM1_af'] = PBRM1_af
    data_dict_tmp['ZFHX3_af'] = ZFHX3_af
    data_dict_tmp['MET_af'] = MET_af
    data_dict_tmp['TP53_af'] = TP53_af
    
    data_dict[ptid] = data_dict_tmp

In [ ]:
data_dict = {}
for ptid in df_allen_mut5['PATIENT_ID'].unique():
    df_tmp1 = df_allen_mut5[df_allen_mut5['PATIENT_ID']==ptid]
    TP53_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='TP53'])
    KEAP1_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1'])
    SMARCA4_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='SMARCA4'])
    STK11_count = len(df_tmp1[df_tmp1['Hugo_Symbol']=='STK11'])
    tmb = len(df_tmp1)
    TP53_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='TP53']['af'].max()
    KEAP1_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='KEAP1']['af'].max()
    SMARCA4_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='SMARCA4']['af'].max()
    STK11_af =  df_tmp1[df_tmp1['Hugo_Symbol']=='STK11']['af'].max()
    
    data_dict_tmp ={}
    data_dict_tmp['TP53_count'] = TP53_count
    data_dict_tmp['KEAP1_count'] = KEAP1_count
    data_dict_tmp['SMARCA4_count'] = SMARCA4_count
    data_dict_tmp['STK11_count'] = STK11_count
    data_dict_tmp['tmb'] = tmb
    data_dict_tmp['TP53_af'] = TP53_af
    data_dict_tmp['KEAP1_af'] = KEAP1_af
    data_dict_tmp['SMARCA4_af'] = SMARCA4_af
    data_dict_tmp['STK11_af'] = STK11_af
    
    data_dict[ptid] = data_dict_tmp

In [297]:
df_test_data = pd.DataFrame.from_dict(data_dict, orient='index').reset_index()

In [298]:
df_test_data = df_test_data.fillna(0)

In [299]:
df_test_data = df_test_data.rename(columns={'index':'PtID'})

In [300]:
df_test_data.head()

,PtID,KEAP1_count,STK11_count,PBRM1_count,ZFHX3_count,MET_count,TP53_count,tmb,KEAP1_af,STK11_af,PBRM1_af,ZFHX3_af,MET_af,TP53_af
0,luad_mskcc_2015_15,0,0,0,0,0,1,165,0.0,0.0,0.0,0.0,0.0,0.073529
1,luad_mskcc_2015_22,0,0,0,0,0,1,85,0.0,0.0,0.0,0.0,0.0,0.169014
2,luad_mskcc_2015_9,0,0,0,0,0,1,342,0.0,0.0,0.0,0.0,0.0,0.469388
3,CANSEQU01-0100425,0,0,0,0,0,0,213,0.0,0.0,0.0,0.0,0.0,0.000000
4,luad_mskcc_2015_18,0,0,0,0,0,0,263,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
df_test_data['recist_label'] = df_test_data['PtID'].map(lambda x: labels_recist_dict[x])

In [301]:
df_test_data['label'] = df_test_data['PtID'].map(lambda x: labels_dict[x])

In [310]:
df_test_data.head()

,PtID,KEAP1_count,STK11_count,PBRM1_count,ZFHX3_count,MET_count,TP53_count,tmb,KEAP1_af,STK11_af,PBRM1_af,ZFHX3_af,MET_af,TP53_af,label
0,luad_mskcc_2015_15,0,0,0,0,0,1,165,0.0,0.0,0.0,0.0,0.0,0.073529,1.0
1,luad_mskcc_2015_22,0,0,0,0,0,1,85,0.0,0.0,0.0,0.0,0.0,0.169014,1.0
2,luad_mskcc_2015_9,0,0,0,0,0,1,342,0.0,0.0,0.0,0.0,0.0,0.469388,1.0
3,CANSEQU01-0100425,0,0,0,0,0,0,213,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
4,luad_mskcc_2015_18,0,0,0,0,0,0,263,0.0,0.0,0.0,0.0,0.0,0.000000,1.0


In [ ]:
df_test_data.recist_label.value_counts()

In [ ]:
df_test_data.columns

In [302]:
X_test = df_test_data[['tmb', 'KEAP1_count', 'STK11_count', 'PBRM1_count', 'ZFHX3_count',
                   'MET_count', 'TP53_count', 'KEAP1_af', 'STK11_af', 'PBRM1_af',
                   'ZFHX3_af', 'MET_af', 'TP53_af']]
# X_test = df_test_data[['tmb', 'TP53_count', 'KEAP1_count', 'SMARCA4_count', 'STK11_count']]

In [303]:
scaler = preprocessing.MinMaxScaler()
scaler_mod = scaler.fit(X_test)
scaled_X = scaler_mod.transform(X_test)
df_scaled_X = pd.DataFrame(scaled_X, columns=X_test.columns)

In [304]:
X_test = df_scaled_X.values
Y_test = df_test_data['label'].values

In [305]:
Y_predict = model_gbc.predict(X_test)

In [312]:
df_test_data['ccg_prediction'] = Y_predict

In [318]:
df_test_data['OS_MONTHS'] = df_test_data['PtID'].map(lambda x: os_months_dict[x])

In [319]:
df_test_data['CNSR'] = df_test_data['PtID'].map(lambda x: cnsr_dict[x])

In [313]:
df_test_data.head()

,PtID,KEAP1_count,STK11_count,PBRM1_count,ZFHX3_count,MET_count,TP53_count,tmb,KEAP1_af,STK11_af,PBRM1_af,ZFHX3_af,MET_af,TP53_af,label,ccg_prediction
0,luad_mskcc_2015_15,0,0,0,0,0,1,165,0.0,0.0,0.0,0.0,0.0,0.073529,1.0,1.0
1,luad_mskcc_2015_22,0,0,0,0,0,1,85,0.0,0.0,0.0,0.0,0.0,0.169014,1.0,0.0
2,luad_mskcc_2015_9,0,0,0,0,0,1,342,0.0,0.0,0.0,0.0,0.0,0.469388,1.0,0.0
3,CANSEQU01-0100425,0,0,0,0,0,0,213,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0
4,luad_mskcc_2015_18,0,0,0,0,0,0,263,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0


In [306]:
from sklearn.metrics import balanced_accuracy_score

In [307]:
balanced_accuracy_score(Y_test, Y_predict)

0.7

In [308]:
Y_predict_prob = model_gbc.predict_proba(X_test)[:,1]

In [309]:
Y_predict_prob

array([0.77478154, 0.25858395, 0.21632797, 0.63373857, 0.77332734,
       0.40018155, 0.98089111, 0.64452713, 0.22012449, 0.62326719,
       0.61139019, 0.28784994, 0.146954  , 0.52231697, 0.99394859,
       0.53431917, 0.47174799, 0.97875962, 0.18908352, 0.08832344,
       0.5448119 , 0.90284522, 0.97509553, 0.35745566, 0.41267686,
       0.91669179, 0.98412077, 0.63373857, 0.61555674, 0.390104  ,
       0.26634847, 0.31210265, 0.95215567, 0.41099263, 0.4771531 ,
       0.17278516, 0.25081796, 0.13427226, 0.14082688, 0.53431917])

### Survival curve analysis

In [320]:
from lifelines import CoxPHFitter
df_tmp = df_test_data[['OS_MONTHS', 'CNSR', 'tmb', 'ccg_prediction']]
cph = CoxPHFitter()
cph.fit(df_tmp, duration_col='OS_MONTHS', event_col='CNSR')
cph.print_summary()

In [ ]:
        columns=['OS', 'CNSR', 'tmb_status', gene]
        df_temp1 = df_gandara1[columns]
        occurence = df_temp1[gene].values.sum()
        cph = CoxPHFitter()
        cph.fit(df_temp1, duration_col='OS', event_col='CNSR')
        HR = cph.hazard_ratios_[gene]
        p =  cph.summary['p'][gene]
        z =  cph.summary['z'][gene]

## 3. Evaluation on Gandara

In [ ]:
df_oak = pd.read_excel('../data/gandara/41591_2018_134_MOESM3_ESM.xlsx', sheet_name='OAK_Clinical_Data')
df_pop = pd.read_excel('../data/gandara/41591_2018_134_MOESM3_ESM.xlsx', sheet_name='POPLAR_Clinical_Data')
df_mut = pd.read_excel('../data/gandara/41591_2018_134_MOESM3_ESM.xlsx', sheet_name='OAK_POPLAR_btmb_variants')

In [ ]:
df_pop1 = df_pop[(df_pop['BCOR']=='PD') | (df_pop['BCOR']=='PR') | (df_pop['BCOR']=='CR')]

In [ ]:
def get_label(disease_status):
    if disease_status == 'PD':
        label=1
    elif disease_status =='PR':
        label=0
    elif disease_status=='CR':
        label=0
    else:
        label = np.nan
    return label

In [ ]:
df_pop1['label'] = df_pop1['BCOR'].map(lambda x: get_label(x))

In [ ]:
df_pop1 = df_pop1[df_pop1['TRT01P']=='MPDL3280A']

In [ ]:
df_pop1.label.value_counts()

In [ ]:
patient_ids_pop = df_pop1.PtID.unique()

In [ ]:
df_mut_pop = df_mut[df_mut['PtID'].isin(patient_ids_pop)]

In [ ]:
len(df_mut_pop.PtID.unique())

In [ ]:
len(df_mut_pop)

In [ ]:
df_mut_pop.head()

### Creating data based on new genes

In [ ]:
data_dict = {}
for ptid in df_mut_pop.PtID.unique():
    df_tmp1 = df_mut_pop[df_mut_pop['PtID']==ptid]
    KEAP1_count = len(df_tmp1[df_tmp1['gene_name']=='KEAP1'])
    STK11_count = len(df_tmp1[df_tmp1['gene_name']=='STK11'])
    PBRM1_count = len(df_tmp1[df_tmp1['gene_name']=='PBRM1'])
    ZFHX3_count = len(df_tmp1[df_tmp1['gene_name']=='ZFHX3'])
    MET_count = len(df_tmp1[df_tmp1['gene_name']=='MET'])
    TP53_count = len(df_tmp1[df_tmp1['gene_name']=='TP53'])
    
    tmb = len(df_tmp1)
    KEAP1_af =  df_tmp1[df_tmp1['gene_name']=='KEAP1']['af'].max()
    STK11_af =  df_tmp1[df_tmp1['gene_name']=='STK11']['af'].max()
    PBRM1_af =  df_tmp1[df_tmp1['gene_name']=='PBRM1']['af'].max()
    ZFHX3_af =  df_tmp1[df_tmp1['gene_name']=='ZFHX3']['af'].max()
    MET_af =  df_tmp1[df_tmp1['gene_name']=='MET']['af'].max()
    TP53_af =  df_tmp1[df_tmp1['gene_name']=='TP53']['af'].max()
    
    data_dict_tmp ={}
    data_dict_tmp['KEAP1_count'] = KEAP1_count
    data_dict_tmp['STK11_count'] = STK11_count
    data_dict_tmp['PBRM1_count'] = PBRM1_count
    data_dict_tmp['ZFHX3_count'] = ZFHX3_count
    data_dict_tmp['MET_count'] = MET_count
    data_dict_tmp['TP53_count'] = TP53_count
    
    data_dict_tmp['tmb'] = tmb
    data_dict_tmp['KEAP1_af'] = KEAP1_af
    data_dict_tmp['STK11_af'] = STK11_af
    data_dict_tmp['PBRM1_af'] = PBRM1_af
    data_dict_tmp['ZFHX3_af'] = ZFHX3_af
    data_dict_tmp['MET_af'] = MET_af
    data_dict_tmp['TP53_af'] = TP53_af
    
    data_dict[ptid] = data_dict_tmp

In [ ]:
df_data_pop = pd.DataFrame.from_dict(data_dict, orient='index').reset_index()

In [ ]:
df_data_pop.head()

In [ ]:
df_data_pop = df_data_pop.rename(columns={'index':'PtID'})

In [ ]:
ptid_btmb_pop_dict = df_pop1[['PtID', 'btmb']].set_index('PtID').to_dict()['btmb']

In [ ]:
df_data_pop['btmb'] = df_data_pop['PtID'].map(lambda x: ptid_btmb_pop_dict[x])

In [ ]:
df_data_pop = df_data_pop.fillna(0)

In [ ]:
df_data_pop.columns

In [ ]:
df_label_pop = df_pop1[['PtID', 'label']]

In [ ]:
df_data_pop_final = pd.merge(df_data_pop,df_label_pop, on='PtID')

#### test data

In [ ]:
df_data_pop_final.label.value_counts()

In [ ]:
# X_test = df_data_pop_final[['KEAP1_count', 'STK11_count', 'PBRM1_count', 'ZFHX3_count',
#                             'MET_count', 'NTRK3_count', 'KEAP1_af', 'STK11_af', 'PBRM1_af',
#                             'ZFHX3_af', 'MET_af', 'NTRK3_af', 'btmb']]
X_test = df_data_pop_final[['btmb', 'KEAP1_count', 'STK11_count', 'PBRM1_count', 'ZFHX3_count',
                   'MET_count', 'TP53_count', 'KEAP1_af', 'STK11_af', 'PBRM1_af',
                   'ZFHX3_af', 'MET_af', 'TP53_af']]
# X_test = df_data_pop_final[['f1', 'f2', 'f3', 'f4']]
# X_test = df_data_pop_final[['btmb']]

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler_mod = scaler.fit(X_test)
scaled_X = scaler_mod.transform(X_test)
df_scaled_X = pd.DataFrame(scaled_X, columns=X_test.columns)

In [ ]:
X_test = df_scaled_X.values
Y_test = df_data_pop_final['label'].values

In [ ]:
Y_predict = model_gbc.predict(X_test)
Y_prob = model_gbc.predict_proba(X_test)[:,1]

In [ ]:
Y_prob

In [ ]:
Y_predict_threshold = (model_gbc.predict_proba(X_test)[:,1] >= 0.35).astype(bool)

In [ ]:
#Y_tmb_pred = df_data_pop_final['tmb_pred'].values

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(Y_test, Y_predict)

In [ ]:
balanced_accuracy_score(Y_test, Y_predict_threshold)

In [ ]:
from sklearn.metrics import roc_auc_score
Y_prob = model_gbc.predict_proba(X_test)[:,1]
roc_auc_score(Y_test, Y_prob)